In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
home_path = '/Users/vs/'

In [3]:
import sys
sys.path.append('/Users/vs/')
sys.path.append('/Users/vs/mds20_replearning')

In [4]:
import torch
from mds20_replearning.scripts.cpc.pl_model import CPCModel
from mds20_replearning.data.language.load import load_polarity
from torch.optim import Adam

In [5]:
model = CPCModel(vocab_size=20000, emb_dim=620, enc_dim=2400, ar_dim=2400, kernel_size=5, lr=2e-4) 

In [6]:
model

CPCModel(
  (cpc): CPC(
    (encoder): PaperEncoder(
      (embedding): Embedding(20000, 620, padding_idx=0)
      (conv_block): Sequential(
        (0): Conv1d(620, 2400, kernel_size=(5,), stride=(1,))
        (1): ReLU()
        (2): AdaptiveAvgPool1d(output_size=1)
      )
    )
    (ar): AutoRegressiveModel(
      (GRU): GRU(2400, 2400, batch_first=True)
      (Wk): Linear(in_features=2400, out_features=2400, bias=True)
    )
  )
)

In [ ]:
# had problems with loading weights TODO!!!

In [7]:
from mds20_replearning.scripts.cpc.pl_data import PosNegDataModule

In [8]:
import pickle
with open('/Users/vs/Downloads/dictionary(1).pkl', 'rb') as f:
    predefined_vocab_mapping = pickle.load(f)

In [9]:
data_module = PosNegDataModule(128, load_polarity, '/Users/vs/mds20_replearning/data/txt_sentoken',
                               url='http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz', 
                               download_root = '/Users/vs/mds20_replearning/data/',
                               filename = 'review_polarity',
                               w2i_mapping=predefined_vocab_mapping, valid_split=0.3)

In [22]:
from mds20_replearning.classification.model import ClassificationModule

In [23]:
linear_model = ClassificationModule(model.cpc.encoder, freeze=False, lr=2e-4)

In [24]:
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=1)  # set required resources here

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [25]:
trainer.fit(linear_model, datamodule=data_module)


  | Name      | Type                       | Params
---------------------------------------------------------
0 | model     | ClassificationArchitecture | 19.8 M
1 | valid_acc | Accuracy                   | 0     


/Users/vs/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1